In [1]:
import pandas as pd
import numpy as np
import os
import datetime

print(os.getcwd())
print(datetime.datetime.now())

def recursive_file_gen(my_root_dir):
    for root, dirs, files in os.walk(my_root_dir):
        for file in files:
            yield os.path.join(root, file)

/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Post_YoY
2020-04-01 17:37:14.952028


In [3]:
weeks_2019Q4=[datetime.date(2019,11,9)+datetime.timedelta(days=x*7) for x in range(13)]
weeks_2018Q4=[x-datetime.timedelta(days=52*7) for x in weeks_2019Q4]
weeks_2018Q4=[str(x).replace("-","") for x in weeks_2018Q4]

folder_2019_by_week="/home/jian/BigLots/"

Q4_files_2019_all_types=list(recursive_file_gen(folder_2019_by_week))
Q4_files_2019_all_types=[x for x in Q4_files_2019_all_types if "aily" in x]
Q4_files_2019=[]
for week_end_date in weeks_2019Q4:
    file_path=[x for x in Q4_files_2019_all_types if str(week_end_date) in x]
    Q4_files_2019=Q4_files_2019+file_path
    
print(len(Q4_files_2019))
Q4_files_2019=sorted(Q4_files_2019)
Q4_files_2019

13


['/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-11-09/MediaStormDailySales20191112-115210-002.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-11-16/MediaStormDailySales20191119-112232-478.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-11-23/MediaStormDailySales20191126-112901-552.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-11-30/MediaStormDailySales20191203.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-12-07/MediaStormDailySales20191211.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-12-14/MediaStormDailySales20191217-195625-000.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-12-21/MediaStormDailySales20191226-122746-000.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-12-28/MediaStormDailySales20191231-112945-515.txt',
 '/home/jian/BigLots/2020_by_weeks/MediaStorm_2020-01-04/MediaStormDailySales20200107-112859-015.txt',
 '/home/jian/BigLots/2020_by_weeks/MediaStorm_2020-01-11/MediaStormDailySales20200114-115009-14

In [12]:
df_shoppers=pd.DataFrame()
for file in Q4_files_2019:
    df=pd.read_table(file,sep="|",dtype=str,usecols=['location_id','customer_id_hashed','transaction_dt'])
    df=df[pd.notnull(df['customer_id_hashed'])]
    print(datetime.datetime.now(),df['transaction_dt'].min(),df['transaction_dt'].max())
    df['store_type']=np.where(df['location_id']=="6990","online","instore")
    df=df[['store_type','customer_id_hashed']]
    df_shoppers=df_shoppers.append(df)
df_shoppers=df_shoppers.drop_duplicates()

2020-04-01 17:45:05.756688 2019-11-03 2019-11-09
2020-04-01 17:49:59.528633 2019-11-10 2019-11-16
2020-04-01 17:53:34.632207 2019-11-17 2019-11-23
2020-04-01 17:58:51.224041 2019-11-24 2019-11-30
2020-04-01 17:59:51.248656 2019-12-01 2019-12-07
2020-04-01 18:05:49.287241 2019-12-08 2019-12-14
2020-04-01 18:10:26.837789 2019-12-15 2019-12-21
2020-04-01 18:21:16.229495 2019-12-22 2019-12-28
2020-04-01 18:32:17.771645 2019-12-29 2020-01-04
2020-04-01 18:33:08.376907 2020-01-05 2020-01-11
2020-04-01 18:33:54.438697 2020-01-12 2020-01-18
2020-04-01 18:35:05.424043 2020-01-19 2020-01-25
2020-04-01 18:36:41.662573 2020-01-26 2020-02-01


In [13]:
print("df_shoppers['customer_id_hashed'].nunique()",df_shoppers['customer_id_hashed'].nunique())
df_summary_unique_shopper_count=df_shoppers.groupby("store_type")['customer_id_hashed'].nunique().to_frame().reset_index()
df_summary_unique_shopper_count

df_shoppers['customer_id_hashed'].nunique() 10401633


,store_type,customer_id_hashed
0,instore,10379012
1,online,60825


In [11]:
df.head(2)

,location_id,transaction_dt,customer_id_hashed,store_type
0,4322,2019-11-09,b445597aeb04ea0afe60321876a7580f52abaa8e4ef5b0...,instore
1,4557,2019-11-06,23b14d85798fd5580f20d79e32824ad4cd6df31f5effe5...,instore


In [14]:
df_shoppers[pd.isnull(df_shoppers['customer_id_hashed'])]

,store_type,customer_id_hashed


In [15]:
10379012+60825

10439837

In [18]:
df[df['store_type']=="online"].shape

(6642, 2)